# TP1: Algoritmos de búsqueda en Torre de Hanoi
### Integrantes:
* Emiliano Uriel Martino
* Juan Pablo Hagata
* Fausto Juárez Yélamos
* Carolina Perez Omodeo
* Diego José Araujo Arellano

### 1. ¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)

Los PEAS del problema son los siguientes:

* **Performance**: La cantidad de movimientos realizados para ordenar la torre.
* **Environment**: Las torres 1, 2 y 3, y las fichas a mover.
* **Actuators**: La acción de mover una ficha de una torre hacia otra.
* **Sensors**: La forma de detectar el estado de cada torre, tanto la cantidad de fichas como el orden en el que se encuentran.



### 2. ¿Cuáles son las propiedades del entorno de trabajo?

Realizamos la clasificación para las propiedades principales:

* **Totalmente Observable** | ~~Parcialmente Observable~~: El problema se puede ver de forma completa.
* **Determinista** | ~~Estocástico~~: No hay decisiones aleatorias.
* **Episódico** | **Secuencial**: Puede considerarse de ambas formas, ya que puede tomarse una decisión, volver a sensar y tomarse otra; y también puede sensarse una sola vez y tomar todas las decisiones siguientes.
* **Estático** | ~~Dinámico~~: El ambiente no cambia.
* **Discreto** | ~~Continuo~~: Puedo moverme de un estado discreto a otro.
* **Agente Individual** | ~~Multiagente~~: Un solo agente realiza todas las acciones.

### 3. En el contexto de este problema, establezca cuáles son los: estado, espacio de estados, árbol de búsqueda, nodo de búsqueda, objetivo, acción y frontera.

Para la torre de Hanoi, las definiciones son las siguientes:
* **Estado**: El estado es la posición de los discos.
* **Espacio de estados**: El espacio de estados son todos los posibles estados, o sea, las posibles combinaciones válidas de discos.
* **Árbol de búsqueda**: Son las secuencias posibles que llevan del estado inicial al final.
* **Nodo de búsqueda**: Son los estados posibles alcanzables mediante las acciones posibles.
* **Objetivo**: Mover todos los discos de la primera torre a la última.
* **Acción**: Mover un disco de una torre a otra, sin que un disco más grande se ponga sobre uno más pequeño.
* **Frontera**: Son los nodos pendientes de explorar.


### 4. Implemente algún método de búsqueda. Puedes elegir cualquiera menos búsqueda en anchura primero (el desarrollado en clase). Sos libre de elegir cualquiera de los vistos en clases, o inclusive buscar nuevos.

In [1]:
# Paquetes y funciones necesarias para correr el código
from hanoi_states import StatesHanoi, ActionHanoi, ProblemHanoi
from tree_hanoi import NodeHanoi
from aima import PriorityQueue
import tracemalloc
import time


In [2]:
# ---------------- Input Discos  ----------------

# Cantidad de discos a considerar en el problema
ndisks = 5

# ---------------- Problema de Hanoi ----------------

initial_state = StatesHanoi(list(range(ndisks, 0, -1)), [], [], max_disks=ndisks)
goal_state = StatesHanoi([], [], list(range(ndisks, 0, -1)), max_disks=ndisks)

# Se define el problema : ir de estado inicial al final y movimientos entre estados
problem = ProblemHanoi(initial= initial_state, goal = goal_state)

print(f"Estado inicial: {initial_state}")
print(f"Estado objetivo: {goal_state}")

Estado inicial: HanoiState: 5 4 3 2 1 |  | 
Estado objetivo: HanoiState:  |  | 5 4 3 2 1


In [3]:
# ---------------- Algoritmo de búsqueda en profundidad ----------------

def dfs(problem):
  frontier = []
  frontier.append(NodeHanoi(problem.initial))
  explored = set()

  while len(frontier) != 0:
    node = frontier.pop()
    explored.add(node.state)
    if problem.goal_test(node.state):
      last_node = node
      #print("Encontramos la solución!")
      break

    for next_node in node.expand(problem):
      if next_node.state not in explored:
        frontier.append(next_node)
  return last_node, frontier, explored


In [4]:
# ---------------- JSON para utilizar en simulador ----------------

last_node, frontier, explored = dfs(problem)
last_node.generate_solution_for_simulator() 

In [5]:
# ---------------- Resultados del algoritmo DFS ----------------

print(f"Longitud del camino de la solución: {last_node.state.accumulated_cost}")
print(f"Hubo {len(frontier)} nodos de frontera y {len(explored)} nodos explorados.")


Longitud del camino de la solución: 121.0
Hubo 63 nodos de frontera y 122 nodos explorados.


### 5. ¿Qué complejidad en tiempo y memoria tiene el algoritmo elegido?

La complejidad en tiempo y en memoria para el algoritmo de búsqueda en profundidad (teóricamente) es la siguiente:
* **Tiempo**: La complejidad temporal es de $O(V + E)$, donde:

    - $V$ es el número de vértices.
    
    - $E$ es el número de aristas.


* **Memoria**: En el peor de los casos  $O(V)$, consume muy poca memoria.

No obstante, no encuentra la solución más eficiente.


### 6. A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? (Se recomienda correr 10 veces y calcular promedio y desvío estándar de las métricas).

##### Prueba del algoritmo en tiempo:

In [6]:
%%timeit -r 10 -n 100
dfs(problem)

8.01 ms ± 323 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


##### Prueba del algoritmo en memoria:

In [7]:
# ----------------Evaluación del rendimiento ----------------

def evaluate_performance(problem, search_func, heuristic_func=None, **kwargs):
    # Medir tiempo de ejecución
    start_time = time.time()
    if heuristic_func:
        solution, frontier, reached = search_func(problem, heuristic_func, **kwargs)
    else:
        solution, frontier, reached = search_func(problem, **kwargs)
    end_time = time.time()

    # Medir uso de memoria
    tracemalloc.start()
    if heuristic_func:
        solution, frontier, reached = search_func(problem, heuristic_func, **kwargs)
    else:
        solution, frontier, reached = search_func(problem, **kwargs)
    _, memory_peak = tracemalloc.get_traced_memory()
    memory_peak /= 1024*1024  # Convertir a MB
    tracemalloc.stop()

    execution_time = end_time - start_time
    return execution_time, memory_peak, solution, frontier, reached


In [8]:
# ---------------- Resultados de evaluación de rendimiento de búsqueda en profundidad ----------------
execution_time, memory_peak, solution, frontier, reached = evaluate_performance(problem, dfs)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")


Tiempo de ejecución: 0.01 segundos
Máxima memoria ocupada: 0.25 MB


### 7. Si la solución óptima es $2^k -1$ movimientos con $k$ igual al número de discos. Qué tan lejos está la solución del algoritmo implementado de esta solución óptima (se recomienda correr al menos $10$ veces y usar el promedio de trayecto usado).

In [9]:
# ---------------- Función genérica para evaluar rendimiento y costo promedio ----------------
def evaluate_performance_with_repeats(problem, search_func, repeats=10, heuristic_func=None, **kwargs):
    # Medir tiempo de ejecución y uso de memoria
    execution_time, memory_peak, solution, frontier, reached = evaluate_performance(problem, search_func, heuristic_func, **kwargs)

    # Calcular el costo promedio de las soluciones encontradas
    total_cost = 0
    for _ in range(repeats):
        if heuristic_func:
            solution, _, _ = search_func(problem, heuristic_func, **kwargs)
        else:
            solution, _, _ = search_func(problem, **kwargs)
        if solution:
            total_cost += solution.state.accumulated_cost

    if repeats > 0:
        average_cost = total_cost / repeats
    else:
        average_cost = None

    return execution_time, memory_peak, average_cost, solution, frontier, reached

In [10]:
# ---------------- Resumen rendimiento de búsqueda en profundidad ----------------

repeats = 10
execution_time, memory_peak, average_cost, solution, frontier, reached = evaluate_performance_with_repeats(
  problem, dfs, repeats)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")
if average_cost is not None:
  print(f"Longitud promedio del camino de la solución: {average_cost}")
else:
  print("No se encontraron soluciones en las repeticiones.")
print(f"El costo promedio de {repeats} repeticiones es {average_cost}, mientras que el costo óptimo es de {2**(ndisks)-1}.")


Tiempo de ejecución: 0.01 segundos
Máxima memoria ocupada: 0.25 MB
Longitud promedio del camino de la solución: 121.0
El costo promedio de 10 repeticiones es 121.0, mientras que el costo óptimo es de 31.


In [11]:
# ---------------- Metodo para imprimir el camino de la solucion ----------------
def print_solution(solution):
  if solution:
    current_solution = solution
    actions = []
    nodes = []
    while current_solution:
      actions.append(current_solution.action)
      nodes.append(current_solution.state)
      current_solution = current_solution.parent
    actions.reverse()
    nodes.reverse()
    for action, node in zip(actions, nodes):
      print(action)
      print(node)
  else:
    print("No se encontró solución.")

In [12]:
# ---------------- Imprimo la solucion de DFS ----------------
print_solution(solution)

None
HanoiState: 5 4 3 2 1 |  | 
Move disk 1 from 1 to 3
HanoiState: 5 4 3 2 |  | 1
Move disk 1 from 3 to 2
HanoiState: 5 4 3 2 | 1 | 
Move disk 2 from 1 to 3
HanoiState: 5 4 3 | 1 | 2
Move disk 1 from 2 to 3
HanoiState: 5 4 3 |  | 2 1
Move disk 1 from 3 to 1
HanoiState: 5 4 3 1 |  | 2
Move disk 2 from 3 to 2
HanoiState: 5 4 3 1 | 2 | 
Move disk 1 from 1 to 3
HanoiState: 5 4 3 | 2 | 1
Move disk 1 from 3 to 2
HanoiState: 5 4 3 | 2 1 | 
Move disk 3 from 1 to 3
HanoiState: 5 4 | 2 1 | 3
Move disk 1 from 2 to 3
HanoiState: 5 4 | 2 | 3 1
Move disk 1 from 3 to 1
HanoiState: 5 4 1 | 2 | 3
Move disk 2 from 2 to 3
HanoiState: 5 4 1 |  | 3 2
Move disk 1 from 1 to 3
HanoiState: 5 4 |  | 3 2 1
Move disk 1 from 3 to 2
HanoiState: 5 4 | 1 | 3 2
Move disk 2 from 3 to 1
HanoiState: 5 4 2 | 1 | 3
Move disk 1 from 2 to 3
HanoiState: 5 4 2 |  | 3 1
Move disk 1 from 3 to 1
HanoiState: 5 4 2 1 |  | 3
Move disk 3 from 3 to 2
HanoiState: 5 4 2 1 | 3 | 
Move disk 1 from 1 to 3
HanoiState: 5 4 2 | 3 | 1
Move d

### Anexo: Implementación y resultados de evaluación de otros algoritmos de búsqueda

#### Algoritmos de búsqueda no informada

##### Algoritmo de búsqueda en profundidad limitada

In [13]:
# ---------------- Algoritmo de búsqueda en profundidad limitada ----------------

def depth_limited_search(problem, limit):
  def recursive_dls(node, problem, limit):
    nonlocal explored, reached
    explored.add(node.state)
    if problem.goal_test(node.state):
      return node
    elif limit == 0:
      return 'cutoff'
    else:
      cutoff_ocurred = False
      for child in node.expand(problem):
        if child.state not in reached:
          reached.add(child.state)
          result = recursive_dls(child, problem, limit - 1)
          if result == 'cutoff':
            cutoff_ocurred = True
          elif result is not None:
            return result
      return 'cutoff' if cutoff_ocurred else None

  explored = set()
  reached = set()
  last_node = recursive_dls(NodeHanoi(problem.initial), problem, limit)
  return last_node, explored, reached


In [14]:
%%time
last_node, explored, reached = depth_limited_search(problem, 65)
if isinstance(last_node, NodeHanoi):
  print(f"Longitud del camino de la solución: {last_node.state.accumulated_cost}")
  print(f"Hubo {len(reached)} nodos alcanzados y {len(explored)} nodos explorados.")
elif last_node == 'cutoff':
  print("Se alcanzó el límite de profundidad sin encontrar una solución.")
  print(f"Hubo {len(reached)} nodos alcanzados y {len(explored)} nodos explorados.")
else:
  print("No se encontró solución.")
  print(f"Hubo {len(reached)} nodos alcanzados y {len(explored)} nodos explorados.")

Longitud del camino de la solución: 65.0
Hubo 164 nodos alcanzados y 165 nodos explorados.
CPU times: total: 0 ns
Wall time: 23.6 ms


In [15]:
# ---------------- Evaluar rendimiento de búsqueda en profundidad limitada ----------------
execution_time, memory_peak, solution, frontier, reached = evaluate_performance(problem, depth_limited_search, limit=65)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")

Tiempo de ejecución: 0.02 segundos
Máxima memoria ocupada: 0.36 MB


In [16]:
# ---------------- Evaluar rendimiento de busqueda en profundidad limitada ----------------
repeats = 10
execution_time, memory_peak, average_cost, solution, frontier, reached = evaluate_performance_with_repeats(
  problem, depth_limited_search, repeats, limit=65)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")
if average_cost is not None:
  print(f"Longitud promedio del camino de la solución: {average_cost}")
else:
  print("No se encontraron soluciones en las repeticiones.")
print(f"El costo promedio de {repeats} repeticiones es {average_cost}, mientras que el costo óptimo es de {2**(ndisks)-1}.")

Tiempo de ejecución: 0.01 segundos
Máxima memoria ocupada: 0.34 MB
Longitud promedio del camino de la solución: 65.0
El costo promedio de 10 repeticiones es 65.0, mientras que el costo óptimo es de 31.


##### Algoritmo de busqueda en profundidad limitada con profundidad iterativa

In [17]:
# ---------------- Algoritmo de búsqueda en profundidad limitada con profundidad iterativa ----------------
def iterative_deepening_search(problem, max_depth=1000):
  for depth in range(0, max_depth):
    result, explored, reached = depth_limited_search(problem, depth)
    if result != 'cutoff':
      return result, explored, reached
  return None, explored, reached

In [18]:
%%time
last_node, explored, reached = iterative_deepening_search(problem, 1000)
if isinstance(last_node, NodeHanoi):
  print(f"Longitud del camino de la solución: {last_node.state.accumulated_cost}")
  print(f"Hubo {len(reached)} nodos alcanzados y {len(explored)} nodos explorados.")
else:
  print("No se encontró solución.")
  print(f"Hubo {len(reached)} nodos alcanzados y {len(explored)} nodos explorados.")

Longitud del camino de la solución: 65.0
Hubo 164 nodos alcanzados y 165 nodos explorados.
CPU times: total: 219 ms
Wall time: 405 ms


In [19]:
# ---------------- Evaluar rendimiento de búsqueda en profundidad limitada con profundidad iterativa ----------------
execution_time, memory_peak, solution, frontier, reached = evaluate_performance(problem, iterative_deepening_search, max_depth=1000)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")

Tiempo de ejecución: 0.43 segundos
Máxima memoria ocupada: 1.42 MB


In [20]:
# ---------------- Evaluar rendimiento de busqueda en profundidad limitada con profundidad iterativa ----------------
repeats = 10
execution_time, memory_peak, average_cost, solution, frontier, reached = evaluate_performance_with_repeats(
  problem, iterative_deepening_search, repeats, max_depth=1000)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")
if average_cost is not None:
  print(f"Longitud promedio del camino de la solución: {average_cost}")
else:
  print("No se encontraron soluciones en las repeticiones.")
print(f"El costo promedio de {repeats} repeticiones es {average_cost}, mientras que el costo óptimo es de {2**(ndisks)-1}.")

Tiempo de ejecución: 0.38 segundos
Máxima memoria ocupada: 1.06 MB
Longitud promedio del camino de la solución: 65.0
El costo promedio de 10 repeticiones es 65.0, mientras que el costo óptimo es de 31.


#### Algoritmos de búsqueda informada

##### Búsqueda voraz (greedy)

In [21]:
# ---------------- Algoritmo de búsqueda informada, búsqueda voraz (greedy) primero el mejor ----------------

def hanoi_heuristic(node, goal_state):
  # Heurística: cuenta los discos en la posición correcta (rod correcta)
  return -sum(1 for rod, goal_rod in zip(node.state.rods, goal_state.rods) if rod == goal_rod)

def greedy_best_first_graph_search(problem, heuristic_func):
    node = NodeHanoi(problem.initial)
    if problem.goal_test(node.state):
        return node, [], set()

    frontier = PriorityQueue('min', f=lambda n: heuristic_func(n, problem.goal))
    frontier.append(node)
    reached = set()
    reached.add(tuple(map(tuple, node.state.rods)))

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node, frontier, reached

        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            state_tuple = tuple(map(tuple, child.state.rods))
            if state_tuple not in reached:
                reached.add(state_tuple)
                frontier.append(child)

    return None, frontier, reached

In [22]:
solution, frontier, reached = greedy_best_first_graph_search(problem, hanoi_heuristic)

if isinstance(solution, NodeHanoi):
  print(f"Longitud del camino de la solución: {solution.state.accumulated_cost}")
  print(f"Hubo {len(frontier)} nodos en la frontera y {len(reached)} nodos explorados.")
else:
  print("No se encontró solución.")
  print(f"Hubo {len(reached)} nodos alcanzados y {len(explored)} nodos explorados.")

Longitud del camino de la solución: 31.0
Hubo 18 nodos en la frontera y 213 nodos explorados.


In [23]:
# ---------------- Evaluar rendimiento de búsqueda informada, búsqueda voraz (greedy) primero el mejor ----------------
execution_time, memory_peak, solution, frontier, reached = evaluate_performance(problem, greedy_best_first_graph_search, heuristic_func=hanoi_heuristic)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")

Tiempo de ejecución: 0.01 segundos
Máxima memoria ocupada: 0.24 MB


In [24]:
# ---------------- Evaluar rendimiento de búsqueda informada, búsqueda voraz (greedy) primero el mejor ----------------
repeats = 10
execution_time, memory_peak, average_cost, solution, frontier, reached = evaluate_performance_with_repeats(
  problem, greedy_best_first_graph_search, repeats, heuristic_func=hanoi_heuristic)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")
if average_cost is not None:
  print(f"Longitud promedio del camino de la solución: {average_cost}")
else:
  print("No se encontraron soluciones en las repeticiones.")
print(f"El costo promedio de {repeats} repeticiones es {average_cost}, mientras que el costo óptimo es de {2**(ndisks)-1}.")

Tiempo de ejecución: 0.01 segundos
Máxima memoria ocupada: 0.22 MB
Longitud promedio del camino de la solución: 31.0
El costo promedio de 10 repeticiones es 31.0, mientras que el costo óptimo es de 31.


##### Búsqueda A*

In [25]:
# ---------------- Algoritmo de búsqueda informada, búsqueda A* ----------------
def hanoi_heuristic(node, goal_state):
    return -sum(1 for rod, goal_rod in zip(node.state.rods, goal_state.rods) if rod == goal_rod)

def astar_search(problem, heuristic_func):
  def f(new_node):
    return new_node.path_cost + heuristic_func(new_node, problem.goal)

  node = NodeHanoi(problem.initial)
  if problem.goal_test(node.state):
    return node, [], set()

  frontier = PriorityQueue('min', f=f)
  frontier.append(node)
  reached = set()
  reached.add(tuple(map(tuple, node.state.rods)))
  state_cost = {tuple(map(tuple, node.state.rods)): f(node)}

  while frontier:
    node = frontier.pop()
    if problem.goal_test(node.state):
      return node, frontier, reached

    for action in problem.actions(node.state):
      child = node.child_node(problem, action)
      state_tuple = tuple(map(tuple, child.state.rods))
      child_cost = f(child)

      if state_tuple not in reached or child_cost < state_cost[state_tuple]:
        reached.add(state_tuple)
        frontier.append(child)
        state_cost[state_tuple] = child_cost

  return None, frontier, reached

In [26]:
%%time
solution, frontier, reached = astar_search(problem, hanoi_heuristic)

if isinstance(solution, NodeHanoi):
  print(f"Longitud del camino de la solución: {solution.state.accumulated_cost}")
  print(f"Hubo {len(frontier)} nodos en la frontera y {len(reached)} nodos explorados.")
else:
  print("No se encontró solución.")
  print(f"Hubo {len(reached)} nodos alcanzados y {len(explored)} nodos explorados.")


Longitud del camino de la solución: 31.0
Hubo 19 nodos en la frontera y 219 nodos explorados.
CPU times: total: 15.6 ms
Wall time: 16.4 ms


In [27]:
# ---------------- Evaluar rendimiento de busqueda informada, busqueda A* ----------------
execution_time, memory_peak, solution, frontier, reached = evaluate_performance(problem, astar_search, heuristic_func=hanoi_heuristic)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")

Tiempo de ejecución: 0.01 segundos
Máxima memoria ocupada: 0.22 MB


In [28]:
# ---------------- Evaluar rendimiento de busqueda informada, busqueda A* ----------------
repeats = 10
execution_time, memory_peak, average_cost, solution, frontier, reached = evaluate_performance_with_repeats(
  problem, astar_search, repeats, heuristic_func=hanoi_heuristic)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Máxima memoria ocupada: {memory_peak:.2f} MB")
if average_cost is not None:
  print(f"Longitud promedio del camino de la solución: {average_cost}")
else:
  print("No se encontraron soluciones en las repeticiones.")
print(f"El costo promedio de {repeats} repeticiones es {average_cost}, mientras que el costo óptimo es de {2**(ndisks)-1}.")

Tiempo de ejecución: 0.02 segundos
Máxima memoria ocupada: 0.22 MB
Longitud promedio del camino de la solución: 31.0
El costo promedio de 10 repeticiones es 31.0, mientras que el costo óptimo es de 31.


In [29]:
# ---------------- Imprimo la solucion de A* ----------------
print_solution(solution)

None
HanoiState: 5 4 3 2 1 |  | 
Move disk 1 from 1 to 3
HanoiState: 5 4 3 2 |  | 1
Move disk 2 from 1 to 2
HanoiState: 5 4 3 | 2 | 1
Move disk 1 from 3 to 2
HanoiState: 5 4 3 | 2 1 | 
Move disk 3 from 1 to 3
HanoiState: 5 4 | 2 1 | 3
Move disk 1 from 2 to 1
HanoiState: 5 4 1 | 2 | 3
Move disk 2 from 2 to 3
HanoiState: 5 4 1 |  | 3 2
Move disk 1 from 1 to 3
HanoiState: 5 4 |  | 3 2 1
Move disk 4 from 1 to 2
HanoiState: 5 | 4 | 3 2 1
Move disk 1 from 3 to 2
HanoiState: 5 | 4 1 | 3 2
Move disk 2 from 3 to 1
HanoiState: 5 2 | 4 1 | 3
Move disk 1 from 2 to 1
HanoiState: 5 2 1 | 4 | 3
Move disk 3 from 3 to 2
HanoiState: 5 2 1 | 4 3 | 
Move disk 1 from 1 to 3
HanoiState: 5 2 | 4 3 | 1
Move disk 2 from 1 to 2
HanoiState: 5 | 4 3 2 | 1
Move disk 1 from 3 to 2
HanoiState: 5 | 4 3 2 1 | 
Move disk 5 from 1 to 3
HanoiState:  | 4 3 2 1 | 5
Move disk 1 from 2 to 1
HanoiState: 1 | 4 3 2 | 5
Move disk 2 from 2 to 3
HanoiState: 1 | 4 3 | 5 2
Move disk 1 from 1 to 3
HanoiState:  | 4 3 | 5 2 1
Move disk